In [41]:
# Python
import pandas as pd
import numpy as np
#
from pathlib import Path
from datetime import datetime
#
import xgboost
#
import importlib
import utilities.train_test.train_test as train_test
import utilities.comparison as comparison

In [42]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

min_date = pd.to_datetime(df.index.max()) - pd.DateOffset(months=12*15)
min_datestr = min_date.strftime('%Y-%m-%d')

#df = df.loc[df.index >= min_datestr]

In [43]:
importlib.reload(train_test)
 
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

file=Path("../../../data/df_tabular_multi.csv")

if file.exists():
    df_tabular_multi=pd.read_csv(file, index_col=0)
else:
    importlib.reload(train_test)
    df_tabular_multi = train_test.get_dataframe_tabular_multi(df)
    df_tabular_multi.to_csv(file)

## Direct forecasting

In [44]:
# True values
y_true = df.mean(axis=1)

## 1 Month

In [45]:
importlib.reload(train_test)

X_train, y_train, X_test, y_test, min_datestr = train_test.get_train_test(df_tabular_multi, months=60)

model_1m = xgboost.XGBRegressor(n_estimators=1000)
model_1m.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [46]:
importlib.reload(comparison)
months_1m = 1
X_train_1m, y_train_1m, X_test_1m, y_test_1m, min_datestr = train_test.get_train_test(df_tabular_multi, months=1)
# reset index
X_train_1m.reset_index(drop=True, inplace=True)
y_train_1m.reset_index(drop=True, inplace=True)
X_test_1m.reset_index(drop=True, inplace=True)
y_test_1m.reset_index(drop=True, inplace=True)

In [47]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 6 months
X_train_input_1m = X_train_1m.head(len(X_train_1m) - len(df.columns) * months_1m)
# Get last 6 months of the training dataset and use as input to predict the next 6 months
X_test_input_1m = X_train_1m.tail(len(df.columns) * months_1m)

# Trained 
y_train_pred_1m = model_1m.predict(X_train_input_1m)
# Predictions
y_test_pred_1m = model_1m.predict(X_test_input_1m)

### 1 month Actual vs Prediction

In [48]:
importlib.reload(comparison)
y_train_mean_pred_1m, y_test_mean_pred_1m = comparison.get_train_test_mean_pred(y_train_pred_1m, y_test_pred_1m,
                                                                                len(df.columns))
comparison.generate_plot(df, df_tabular_multi, y_train_mean_pred_1m, y_test_mean_pred_1m)

286 285 1


#### Overview table

In [49]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_1m_list = y_train_1m['m_return_target(t+1)'].tolist()
y_test_1m_list = y_test_pred_1m.tolist()
#
df_to_evaluate_1m = comparison.get_df_from_pred_list(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_1m = df_to_evaluate_1m - 1
df_to_evaluate_1m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.060000,...,0.00000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.000000,0.000000,-0.010000
1,0.000000,0.000000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.050000,...,0.00000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.000000,0.000000,0.090000
2,0.010000,0.000000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.060000,...,0.00000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.240000,0.000000,0.050000
3,0.000000,-0.030000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.080000,...,-0.48000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.000000,0.000000,0.040000
4,0.000000,0.000000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.020000,...,1.18000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.000000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.100000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.060000,0.060000,...,0.09000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.030000,-0.020000,-0.140000,-0.020000
283,0.000000,-0.040000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.050000,0.110000,...,0.07000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.190000,0.010000,0.140000,-0.020000
284,0.190000,0.080000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.070000,-0.080000,...,0.10000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.140000,0.280000,-0.100000,0.090000
285,-0.040000,-0.220000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.100000,0.060000,...,-0.05000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.030000,-0.070000,-0.110000,0.040000


In [63]:
importlib.reload(comparison)
df_pred_1m, raw_weights_1m, mu_1m, S_1m, sigma_1m, sharpe_1m = comparison.get_portfolio_performance(df_to_evaluate_1m,
                                                                                                    "xgbooster_weights_1m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)

allocation, leftover = comparison.create_discrete_allocation(df_pred_1m, raw_weights_1m)

# Create overview
df_view_1m = pd.DataFrame.from_dict(raw_weights_1m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_1m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_1m)), index=S_1m.columns).values
# Set annual returns
df_view_1m['avg_annual_return'] = mu_1m.values
#
df_view_1m['return_last_period(1m)'] = round(df.tail(1).prod() - 1, 2)
#
df_view_1m

Expected annual return: 66.7%
Annual volatility: 9.4%
Sharpe Ratio: 6.88
Discrete allocation: {'RS1.L': 45800, 'KE': 25, 'TEG.DE': 3515, 'ACCO': 10, '7912.T': 1066, 'KEYS': 11, 'SGRO.L': 17, 'CBRE': 18, 'BRC': 41, 'BBOX.L': 47, 'REL.L': 284, 'OLED': 3, 'CDW': 5, 'FLEX': 181, 'REZI': 9, 'RYN': 32, 'CMPR': 7, 'DLAR.L': 386, '2379.T': 5, '6754.T': 3, 'TTMI': 12, '3234.T': 88, 'BYG.L': 433, 'ASGN': 7, 'COA.L': 31, 'VVV': 6, 'PAGE.L': 10, 'GROW.L': 4, 'KFY': 4, 'MEI': 11, 'RGLD': 11, 'IWG.L': 6, 'ZIL2.DE': 11, 'SANM': 9, '7893.T': 8, 'CTS': 7, 'MAN': 18, '7994.T': 48, '3283.T': 11, 'PDM': 64, 'III.L': 1953, 'NBS.L': 7, 'DSCV.L': 3, '6183.T': 3, '4384.T': 8, 'BRBY.L': 6, '7984.T': 1234, 'CCK': 11, 'APTV': 3, 'STEM.L': 43, 'UU.L': 716, 'ITRI': 29, '6287.T': 56157, 'INF.L': 23, '4218.T': 132, 'WDC': 25, '7313.T': 69, 'EBF': 11, 'TNET': 5, '6098.T': 1115, 'MLKN': 56, 'HLMA.L': 7, 'WPP.L': 9, 'JBL': 9, '3279.T': 9, 'JEN.DE': 49, 'LXP': 7, '6028.T': 5, 'NLY': 23, '7846.T': 442, 'KRG': 218, 'ELME'

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(1m)
RS1.L,0.00000,1.574895,0.091108,-0.01
KE,0.00000,1.627633,-0.308787,-0.05
TEG.DE,0.00223,1.604682,0.491010,0.10
LEG.DE,0.00000,1.675096,0.345630,0.08
SCS,0.00000,1.596705,0.296699,-0.07
...,...,...,...,...
KVHI,0.00000,1.505115,-0.096960,0.00
MOON.L,0.00000,1.579465,0.306064,-0.05
NEO,0.00000,1.935046,0.402915,-0.05
6055.T,0.00000,1.745802,-0.179179,-0.06


## 6 Months


In [51]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test, min_datestr = train_test.get_train_test(df_tabular_multi, months=60, target_key='m_return_target(t+6)')

# 
model_6m = xgboost.XGBRegressor(n_estimators=1000)
model_6m.fit(X_train, y_train,
             eval_set=[(X_train, y_train), (X_test, y_test)],
             verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [52]:
months_6m = 6
X_train_6m, y_train_6m, X_test_6m, y_test_6m, min_datestr = train_test.get_train_test(df_tabular_multi, months=6,
                                                                                      target_key='m_return_target(t+6)')

In [53]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 6 months
X_train_input_6m = X_train_6m.head(len(X_train_6m) - len(df.columns)*months_6m)
# Get last 6 months of the training dataset and use as input to predict the next 6 months
X_test_input_6m = X_train_6m.tail(len(df.columns)*months_6m)

# Trained 
y_train_pred_6m = model_6m.predict(X_train_input_6m)
# Predictions
y_test_pred_6m = model_6m.predict(X_test_input_6m)

### 6 Months Actual vs Prediction

In [54]:
importlib.reload(comparison)
y_train_mean_pred_6m, y_test_mean_pred_6m = comparison.get_train_test_mean_pred(y_train_pred_6m, y_test_pred_6m, len(df.columns))
#
comparison.generate_plot(df, df_tabular_multi, y_train_mean_pred_6m, y_test_mean_pred_6m)

281 275 6


#### Overview

In [55]:
# Get train true values followed with predicted month/s
y_train_6m_list = y_train_6m['m_return_target(t+6)'].tolist()
y_pred_6m_list = y_test_pred_6m.tolist()
#
df_to_evaluate_6m = comparison.get_df_from_pred_list(df, y_train_1m_list, y_pred_6m_list)
df_to_evaluate_6m = df_to_evaluate_6m - 1
df_to_evaluate_6m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.060000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.000000,0.000000,-0.010000
1,0.000000,0.000000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.050000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.000000,0.000000,0.090000
2,0.010000,0.000000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.060000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.240000,0.000000,0.050000
3,0.000000,-0.030000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.080000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.000000,0.000000,0.040000
4,0.000000,0.000000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.020000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.000000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0.207909,0.144162,-0.043117,-0.259218,0.061998,-0.079641,-0.028286,-0.236143,0.048966,-0.095278,...,0.028721,-0.030545,-0.011459,-0.062361,0.025314,-0.059249,-0.050485,-0.075986,0.002439,-0.037006
288,0.031809,-0.050662,-0.291548,0.169990,0.232816,0.106582,0.041020,-0.011095,0.142822,0.459176,...,0.014214,-0.097049,0.022515,-0.136759,0.031766,-0.085253,0.050235,-0.060957,0.024481,-0.083165
289,-0.044819,-0.005423,0.158057,-0.145466,-0.202455,0.062804,0.272185,0.398449,0.006437,-0.027969,...,0.002959,-0.005339,0.039463,-0.019098,-0.014503,-0.050913,-0.040114,0.060692,-0.007348,-0.002113
290,-0.018310,-0.236451,0.081958,-0.267170,-0.216069,-0.145824,0.500675,-0.027654,-0.092588,0.002908,...,0.001781,-0.001685,0.020563,0.034564,-0.009830,0.070898,-0.025842,0.010975,-0.000774,-0.034017


In [56]:
importlib.reload(comparison)
df_pred_6m, raw_weights_6m, mu_6m, S_6m, sigma_6m, sharpe_6m = comparison.get_portfolio_performance(df_to_evaluate_6m,
                                                                                                    "xgbooster_weights_6m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)
comparison.create_discrete_allocation(df_pred_6m, raw_weights_6m)

# Create overview
df_view_6m = pd.DataFrame.from_dict(raw_weights_6m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_6m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_6m)), index=S_6m.columns).values
# Set annual returns
df_view_6m['avg_annual_return'] = mu_6m.values
#
df_view_6m['return_last_period(6m)'] = round(df.tail(6).prod() - 1, 2)
df_view_6m

Expected annual return: 134.6%
Annual volatility: 13.4%
Sharpe Ratio: 9.89
Discrete allocation: {'RS1.L': 1414, 'KE': 1303, 'LEG.DE': 595, 'SCS': 1273, 'HNI': 239, 'KEYS': 227, 'PGRE': 195, 'PSON.L': 35, 'REL.L': 43255, 'TMV.DE': 36, 'BHE': 27, 'PLXS': 7699, 'GPE.L': 4148, 'HAS': 1219, 'JLL': 17, 'LAND.L': 30, 'CDW': 470, 'REZI': 15, 'ARW': 54374, 'FN': 28825, 'RYN': 145, 'CMPR': 12, 'DLAR.L': 861, '2379.T': 87, '6754.T': 745, 'SHA.DE': 49, 'TTMI': 14, '3234.T': 34, 'PBI': 442, 'BYG.L': 41982, 'ASGN': 166, 'COA.L': 3778, 'VVV': 25, 'PAGE.L': 26, 'DB1.DE': 50, 'AVB': 339, 'STWD': 8, 'MTO.L': 1522, 'KFY': 970, 'MEI': 165, 'KFRC': 47, '8954.T': 307, 'ZIL2.DE': 312, 'SANM': 14098, 'YETI': 829, 'ACN': 600, 'HPP': 90, '7893.T': 52844, 'SVS.L': 572, 'AGNC': 141, 'FUTR.L': 2918, 'EBOX.L': 25366, 'III.L': 677, '6183.T': 22, 'KELYA': 54, '6724.T': 187, 'ST': 58, 'BRBY.L': 80, 'CCK': 39532, 'APTV': 282, 'ITRI': 17, 'INF.L': 498, '4218.T': 51, 'HMSO.L': 12, '7313.T': 5, 'TNET': 13, 'WPP.L': 61, 'J

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(6m)
RS1.L,0.00088,2.251590,0.159968,0.09
KE,0.00218,2.315766,-0.397360,-0.18
TEG.DE,0.00045,2.326665,-0.082227,0.28
LEG.DE,0.00257,2.398666,-0.190825,0.22
SCS,0.00086,2.354566,-0.146229,0.02
...,...,...,...,...
KVHI,0.00000,2.203830,-0.230992,-0.12
MOON.L,0.00070,2.231290,0.165106,0.18
NEO,0.00000,2.277950,-0.106647,-0.00
6055.T,0.00000,2.220148,-0.311588,-0.31


## 12 Months

In [57]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test, min_datestr = train_test.get_train_test(df_tabular_multi, months=60, target_key='m_return_target(t+12)')

# 
model_12m = xgboost.XGBRegressor(n_estimators=1000)
model_12m.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_test, y_test)],
              verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [58]:
months_12m = 12
X_train_12m, y_train_12m, X_test_12m, y_test_12m, min_datestr = train_test.get_train_test(df_tabular_multi, months=months_12m,
                                                                                          target_key='m_return_target(t+12)')

In [59]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 12 months
X_train_input_12m = X_train_12m.head(len(X_train_12m) - len(df.columns)*months_12m)
# Get last 6 months of the training dataset and use as input to predict the next 12 months
X_test_input_12m = X_train_12m.tail(len(df.columns)*months_12m)

# Trained 
y_train_pred_12m = model_12m.predict(X_train_input_12m)
# Predictions
y_test_pred_12m = model_6m.predict(X_test_input_12m)

### 12 Months Actual vs Prediction

In [60]:
importlib.reload(comparison)
#
importlib.reload(comparison)
y_train_mean_pred_12m, y_test_mean_pred_12m = comparison.get_train_test_mean_pred(y_train_pred_12m, y_test_pred_12m, len(df.columns))
#
comparison.generate_plot(df, df_tabular_multi, y_train_mean_pred_12m, y_test_mean_pred_12m)

275 263 12


#### Overview table

In [61]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_12m_list = y_train_12m['m_return_target(t+12)'].tolist()
y_test_12m_list = y_test_pred_12m.tolist()
#
df_to_evaluate_12m = comparison.get_df_from_pred_list(df, y_train_12m_list, y_test_12m_list)
df_to_evaluate_12m = df_to_evaluate_12m - 1
df_to_evaluate_12m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,0.320000,0.000000,0.050000,0.100000,0.140000,0.000000,0.000000,0.140000,...,0.000000,0.000000,0.000000,0.040000,0.000000,0.530000,0.000000,0.000000,0.010000,0.110000
1,0.000000,0.000000,0.050000,-0.020000,0.090000,0.060000,0.150000,1.430000,-0.020000,0.010000,...,0.000000,0.000000,0.000000,0.050000,0.000000,-0.170000,0.000000,0.000000,0.000000,0.060000
2,0.000000,0.000000,-0.150000,0.000000,0.040000,0.090000,0.070000,0.000000,0.000000,-0.010000,...,-0.120000,0.000000,0.000000,0.020000,0.000000,-0.010000,0.000000,0.000000,0.000000,0.040000
3,0.010000,0.000000,0.040000,0.000000,0.070000,0.010000,0.050000,-0.650000,0.000000,-0.100000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.230000,0.000000,-0.050000,-0.010000,0.090000
4,0.000000,0.000000,-0.020000,0.000000,-0.030000,-0.030000,-0.010000,0.060000,0.000000,0.100000,...,0.010000,-0.100000,0.000000,-0.030000,0.000000,0.020000,0.000000,0.000000,0.010000,-0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.067907,-0.021461,-0.002982,0.091579,0.086595,0.039283,0.043412,0.056234,0.024255,-0.060713,...,0.025092,0.054947,-0.011317,0.020711,-0.064203,0.028721,-0.011459,0.025314,-0.050485,0.002439
283,0.036402,0.025391,0.052831,0.055507,-0.065196,0.003361,0.041327,0.035622,0.011203,-0.024679,...,0.037940,0.007989,0.065616,0.056634,-0.001989,0.014214,0.022515,0.031766,0.050235,0.024481
284,0.077426,-0.005296,-0.008887,-0.006048,-0.054363,0.076097,0.008334,0.037440,0.022444,-0.064352,...,0.014857,0.006403,0.041907,0.008563,-0.071166,0.002959,0.039463,-0.014503,-0.040114,-0.007348
285,0.076494,0.078687,-0.003362,-0.004569,-0.009458,0.033539,-0.007898,0.026216,0.008877,-0.029621,...,-0.006199,-0.050273,0.005493,0.015794,0.067014,0.001781,0.020563,-0.009830,-0.025842,-0.000774


In [62]:
importlib.reload(comparison)
df_pred_12m, raw_weights_12m, mu_12m, S_12m, sigma_12m, sharpe_12m = comparison.get_portfolio_performance(df_to_evaluate_12m,
                                                                                                          "xgbooster_weights_12m.csv",
                                                                                                          min_avg_return=-0.5,
                                                                                                          months=12)
comparison.create_discrete_allocation(df_pred_12m, raw_weights_12m)

# Create overview
df_view_12m = pd.DataFrame.from_dict(raw_weights_12m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_12m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_12m)), index=S_12m.columns).values
# Set annual returns
df_view_12m['avg_annual_return'] = mu_12m.values
# 
df_view_12m['return_last_period(12m)'] = round(df.tail(12).prod() - 1, 2)
df_view_12m

Expected annual return: 322.4%
Annual volatility: 13.3%
Sharpe Ratio: 24.12
Discrete allocation: {'LEG.DE': 738, 'ACCO': 788, 'KEYS': 2, 'SGRO.L': 2, 'BRC': 275, 'PGRE': 230, 'PSON.L': 1457, 'BBOX.L': 10, 'REL.L': 5, 'PLXS': 2186, 'JLL': 4168, 'OLED': 17094, 'ARW': 93, 'FN': 13768, 'NOVT': 1079, 'RYN': 325, 'DLAR.L': 44028, 'SHA.DE': 47850, '3234.T': 281, 'HOUS': 57, 'VVV': 10, 'PAGE.L': 21, 'DB1.DE': 96, 'AVB': 139, 'STWD': 938, 'GROW.L': 178, 'KFY': 817, 'MEI': 2553, 'RGLD': 549, 'KFRC': 879, 'IWG.L': 9, '8954.T': 303, 'SANM': 10, 'SVS.L': 1701, 'CTS': 1, 'IPG': 412, 'BMI': 201, 'FUTR.L': 537, 'MAN': 2, '3283.T': 210, 'PDM': 7, 'III.L': 944, 'NBS.L': 9, 'DSCV.L': 1015, 'KELYA': 456, 'UU.L': 36291, '6287.T': 6, 'HMSO.L': 408, 'WDC': 37, '7313.T': 10, '6098.T': 16, 'WPP.L': 154, 'JBL': 37, '3279.T': 327, 'TRMB': 26, '6028.T': 459, 'NLY': 25, 'KGF.L': 106, 'RMAX': 877, 'SBAC': 241, 'ATR': 46, 'ELME': 275, 'TEP.L': 31, 'DLX': 8, 'SNWS.L': 155, 'NWSA': 51, '7914.T': 667, 'EXPN.L': 378, 'G

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(12m)
RS1.L,0.00000,2.119187,-0.272466,0.08
KE,0.00038,2.040427,-0.098751,-0.35
TEG.DE,0.00093,2.049478,-0.242138,0.63
LEG.DE,0.00233,2.155247,0.509632,0.49
SCS,0.00000,2.082879,-0.200005,0.22
...,...,...,...,...
KVHI,0.00142,2.044729,0.044364,-0.12
MOON.L,0.00000,2.040971,-0.141553,0.22
NEO,0.00000,2.047679,-0.210483,0.28
6055.T,0.00000,2.048253,-0.241987,-0.26
